In [ ]:
#Import Libraries
import json
import spotipy
import spotipy.util as util
import pandas as pd
import numpy as np
import time
import random
import csv
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
#assign parameters for authorization
client_id = '122378e2606d48a7bb61507f17d7e3e9'
client_secret = 'adeed656de8642cc895bb6e09e6b1a1e'
username = 'jofussaures'
scope = 'user-read-playback-state user-library-read user-modify-playback-state'
redirect_uri = 'https://localhost:8080/callback/'

token = util.prompt_for_user_token(username, scope, client_id, client_secret, redirect_uri)

#initiates spotipy with token, authorized by user
sp = spotipy.Spotify(auth = token)

In [ ]:
#checks to see what Spotify account gave authorization
def currentUser():
    current_user = sp.current_user()
    return current_user

In [ ]:
#gets the track URI
def currentTrackURL():
    current_track_uri = current_song1['item']['uri']
    return current_track_uri

In [ ]:
def getUerSavedTracks():
    uri = []
    number_of_tracks = 50
    for i in range(0,10000,50):
        all_tracks = sp.current_user_saved_tracks(number_of_tracks, i)
        for item in all_tracks['items']:
            track = item['track']
            uri.append(track['uri'])

In [ ]:
def getTrackFeatures(uri):
    tracks = []
    meta = sp.track(uri)
    features = sp.audio_features(uri)

  # meta
    name = meta['name']
    artists = meta['album']['artists'][0]['name']
    year = meta['album']['release_date'][:4]
    trackUri = meta['uri']


  # features
    acousticness = features[0]['acousticness']
    danceability = features[0]['danceability']
    energy = features[0]['energy']
    instrumentalness = features[0]['instrumentalness']
    key = features[0]['key']
    liveness = features[0]['liveness']
    mode = features[0]['mode']
    popularity = features[0]['popularity']
    loudness = features[0]['loudness']
    speechiness = features[0]['speechiness']
    tempo = features[0]['tempo']
    valence = features[0]['valence']

    track = [name, artists, year, acousticness, danceability, energy, instrumentalness, key, 
             liveness, mode, popularity, loudness, speechiness tempo, valence, uri]
    return track

for i in range(len(uri)):
    time.sleep(.005)
    track = getTrackFeatures(uri[i])
    tracks.append(track)
    
df = pd.DataFrame(tracks, columns = ['name', 'artists', 'year', 'acousticness', 'danceability', 'energy', 'instrumentalness',
                                     'key', 'liveness', 'mode', 'popularity','loudness', 'speechiness', 'tempo', 'valence', 'uri'])
df.to_csv("spotify.csv", sep = ',')

In [ ]:
#Import datasheet with pandas
#rdata = rawdata
df = pd.read_csv('spotify.csv')

In [ ]:
#dfw = dataframe w/o name and artists
da = df['artists']
dn = df['name']
du = df['uri']
dfw=df.drop(columns = ['name','artists'])

In [ ]:
features = dfw[['acousticness','danceability','energy','instrumentalness','loudness','speechiness','tempo','valence']]

In [ ]:
km = KMeans(n_clusters = 4)
km.fit(dfw)
cluster = km.labels_

In [ ]:
newdf=dfw.assign(Cluster=cluster)
grouped=newdf.groupby(["Cluster"])
grouped.agg({'acousticness' : 'mean','danceability' : 'mean','energy' : 'mean','instrumentalness' : 'mean','loudness' : 'mean',
             'speechiness' : 'mean','tempo' : 'mean','valence' : 'mean',})

In [ ]:
newdf_melt=pd.melt(newdf.reset_index(), id_vars =['Cluster'],value_vars=['acousticness','danceability','energy',
                                                                         'instrumentalness','loudness','speechiness','tempo',
                                                                         'valence'],var_name='Metric',value_name='Value')

In [ ]:
c0 = newdf[newdf['Cluster']==0].reindex()
c1 = newdf[newdf['Cluster']==1].reindex()
c2 = newdf[newdf['Cluster']==2].reindex()
c3 = newdf[newdf['Cluster']==3].reindex()

In [ ]:
#add artists and song name back into clusters (at end of DataFrame)
#c#a = cluster# w/ artist
#c#f = cluster# w/ artist and name
#c#f = cluster# w/ full data
c0a = c0.assign(artists=da['artists'])
c0an = c0a.assign(name=dn['name'])
c0f = c0an.assign(uri=du['uri'])

c1a = c1.assign(artists=da['artists'])
c1an = c1a.assign(name=dn['name'])
c1f = c1an.assign(uri=du['uri'])

c2a = c2.assign(artists=da['artists'])
c2an = c2a.assign(name=dn['name'])
c2f = c2an.assign(uri=du['uri'])

c3a = c3.assign(artists=da['artists'])
c3an = c3a.assign(name=dn['name'])
c3f = c3an.assign(uri=du['uri'])

In [ ]:
#puts artists and name at beginning of DataFrame
neworder = ['name','artists','year','acousticness','danceability','energy','instrumentalness','key','liveness',
            'loudness','mode','popularity','speechiness','tempo','valence','Cluster', 'uri']
c0f = c0f.reindex(columns = neworder)
c1f = c1f.reindex(columns = neworder)
c2f = c2f.reindex(columns = neworder)
c3f = c3f.reindex(columns = neworder)

In [ ]:
fourframes = [c0f,c1f,c2f,c3f]
allsongs = pd.concat(fourframes)
allsongs

In [ ]:
#search for currentTrackURI() in allsongs.df

#get currentTrackURI()'s cluster

#search for X# of songs +-5 years of song && in same cluster
     #return song's URI to list

#for each URI in list, add to queue